In [1]:
import numpy as np

def sigmod(x, derive=False):
    if (derive == True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))


In [2]:
x = np.array([[0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [1, 1, 1],
              [0, 0, 1]])
x.shape


(5, 3)

In [3]:
y = np.array([[0], [1], [1], [0], [0]])
y.shape


(5, 1)

In [6]:
np.random.seed(1)
w0 = 2 * np.random.random((3, 4)) - 1
w1 = 2 * np.random.random((4, 1)) - 1
w0, w1
for j in range(60000):
    l0 = x
    l1 = sigmod(np.dot(l0, w0))
    l2 = sigmod(np.dot(l1, w1))
    l2_error = y - l2
    if (j % 10000 == 0):
        print("error:" + str(np.mean(np.abs(l2_error))))
    l2_delta = l2_error * sigmod(l2, derive=True)
    l1_error = l2_delta.dot(w1.T)
    l1_delta = l1_error * sigmod(l1, derive=True)
    w1 += (l1.T).dot(l2_delta)
    w0 += (l0.T).dot(l1_delta)


error:0.4918739397364801


error:0.009794284463972344


error:0.006493076538438727


error:0.005144157678376962


error:0.00437274918169706


error:0.0038603867449685135


In [33]:
class NeuralNetwork:
    def __init__(self, num_classes,
                 input_size, hidden_size,
                 reg=1e-5,
                 learning_rate=1e-4,
                 batch=1000):
        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reg = reg
        self.batch = batch
        self.learning_rate = learning_rate

    def initialize(self):
        self.w0 = 1e-3 * np.random.randn(self.input_size, self.hidden_size)
        self.w1 = 1e-3 * np.random.randn(self.hidden_size, self.num_classes)

    def relu(self, x):
        return np.maximum(0, x)

    def front_propagation(self, x, w):
        score = x.dot(w)
        return score

    def softmax(self, x, y, w0, w1):
        n = x.shape[0]
        correct_classes = x[range(x.shape[0]), list(y)].reshape(-1, 1)
        exp_sum = np.sum(np.exp(x), axis=1).reshape(-1, 1)
        loss = np.sum(np.log(exp_sum) - correct_classes)
        loss = loss / n + \
               0.5 * self.reg * np.sum(w0 ** 2) + \
               0.5 * self.reg * np.sum(w1 ** 2)

        dw = np.exp(x) / exp_sum
        dw[range(n), list(y)] -= 1
        dw /= n
        return loss, dw

    def relu_backpropagation(self, dout, x):
        dx = dout
        dx[x <= 0] = 0
        return dx

    def backpropagation(self, x, w, dout):
        dx = dout.dot(w.T)
        dw = x.T.dot(dout)
        return dx, dw

    def ready_for_data(self, x, y):
        training_x_list = []
        training_y_list = []
        start = 0
        num_train = x.shape[0]
        n = int(num_train / self.batch)
        for i in range(n):
            singleX = x[start:start + self.batch, :]
            singleY = y[start:start + self.batch]
            start += self.batch
            training_x_list.append(singleX)
            training_y_list.append(singleY)
        return training_x_list, training_y_list, n

    def pridict(self, x, y):
        score = x.dot(self.w0)
        score = self.relu(score)
        score = score.dot(self.w1)
        ypredict = np.argmax(score, axis=1)
        acc = np.mean(ypredict == y)
        return acc

    def training(self, x, y, time=50):
        for i in range(time):
            l0 = x
            l1 = self.front_propagation(l0, self.w0)
            l1_relu = self.relu(l1)
            l2 = self.front_propagation(l1_relu, self.w1)
            loss, dout = self.softmax(l2, y, self.w0, self.w1)

            if (i == time - 1):
                for k in range(int(time / 100)):
                    print("=", end="=")
                    pass
                print("| 100.0%")

            if (i % 100 == 0):
                print("loss : {:.6f}".format(loss))
                for k in range(int(i / 100)):
                    print("=", end="=")
                    pass
                print(" ", (i / time) * 100, "%")

            dout, dw1 = self.backpropagation(l1_relu, self.w1, dout)

            dout = self.relu_backpropagation(dout, l1)

            dout, dw0 = self.backpropagation(l0, self.w0, dout)

            dw0 += self.reg * self.w0
            dw1 += self.reg * self.w1

            self.w0 -= self.learning_rate * dw0
            self.w1 -= self.learning_rate * dw1

            pass
        pass


In [30]:
#assignment1-two layer network
import pickle

def getDate(file):
    with open(file, 'rb') as fileOpen:
        dict = pickle.load(fileOpen, encoding="bytes")
    return dict


In [31]:
dict1 = getDate("CS231n/cifar-10-batches-py/data_batch_1")
dict2 = getDate("CS231n/cifar-10-batches-py/data_batch_2")
dict3 = getDate("CS231n/cifar-10-batches-py/data_batch_3")
dict4 = getDate("CS231n/cifar-10-batches-py/data_batch_4")
dict5 = getDate("CS231n/cifar-10-batches-py/data_batch_5")



In [34]:
def run_batch(trainX, trainY, testX, testY, classifier, init=False):
    if (init == True):
        print("初始化weight")
        classifier.initialize()
    train_X, train_Y, n = classifier.ready_for_data(trainX, trainY)
    for i in range(n):
        print("第{0}个batch : ".format(i + 1))
        classifier.training(train_X[i], train_Y[i])
        print("accruacy : ", classifier.pridict(testX, testY))

classifier = NeuralNetwork(10, 3 * 32 * 32, 128)
trainX = dict1[b'data']
trainY = dict1[b'labels']
print("第1组数据 : ")
run_batch(trainX, trainY, dict2[b'data'], dict2[b'labels'], classifier, init=True)

trainX = dict2[b'data']
trainY = dict2[b'labels']
print("第2组数据 : ")
run_batch(trainX, trainY, dict1[b'data'], dict1[b'labels'], classifier)

trainX = dict3[b'data']
trainY = dict3[b'labels']
print("第3组数据 : ")
run_batch(trainX, trainY, dict2[b'data'], dict2[b'labels'], classifier)

trainX = dict4[b'data']
trainY = dict4[b'labels']
print("第4组数据 : ")
run_batch(trainX, trainY, dict3[b'data'], dict3[b'labels'], classifier)


第3组数据 : 
初始化weight
第1个batch : 
loss : 2.308416
  0.0 %


| 100.0%


accruacy :  0.1474
第2个batch : 
loss : 2.261027
  0.0 %


| 100.0%


accruacy :  0.1997
第3个batch : 
loss : 2.185547
  0.0 %


...

| 100.0%


accruacy :  0.3413
第2个batch : 
loss : 1.862158
  0.0 %


| 100.0%


accruacy :  0.3475
第3个batch : 
loss : 1.851304
  0.0 %


KeyboardInterrupt: 

In [104]:
classifier.pridict(dict1[b'data'], dict1[b'labels'])

0.391